In [4]:
%pip install ccxt pandas beautifulsoup4 python-dateutil requests --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import ccxt, pandas as pd
from bs4 import BeautifulSoup

print("ccxt:", ccxt.__version__)
print("pandas:", pd.__version__)
print("BeautifulSoup4 OK")


ccxt: 4.5.6
pandas: 2.1.1
BeautifulSoup4 OK


In [6]:
import re, time, logging, requests
from datetime import datetime, timezone, timedelta
from typing import List, Dict, Tuple, Optional
import pandas as pd
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
import ccxt

LOG = logging.getLogger("top200nb")
if not LOG.handlers:
    logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

UA = "Mozilla/5.0"
CMC_HISTORICAL_ROOT = "https://coinmarketcap.com/historical"

# a few symbol normalizations that bite historically
SYMBOL_NORMALIZE = {
    "MATIC": "POL",
    "MIOTA": "IOTA",
    "BTTOLD": "BTT",
    "BCC": "BCH",         # ancient CMC/Binance alias
    "XBT": "BTC",
}

def month_bounds_utc(now_utc: datetime, months_back: int) -> List[Tuple[datetime, datetime]]:
    first = now_utc.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
    out=[]
    for i in range(months_back):
        start = first - relativedelta(months=i)
        end   = start + relativedelta(months=1)
        out.append((start, end))
    return list(reversed(out))

def first_cmc_snapshot_on_or_after(d: datetime) -> datetime:
    wd = d.weekday()                 # Mon=0 … Sun=6
    offset = (6 - wd) % 7
    return (d + timedelta(days=offset)).replace(hour=0, minute=0, second=0, microsecond=0, tzinfo=timezone.utc)

def _flatten_cols(cols) -> list:
    # Handle MultiIndex or odd headers from read_html
    if isinstance(cols, pd.MultiIndex):
        flat = [" ".join([str(x) for x in tup if str(x) != "nan"]).strip() for tup in cols.tolist()]
    else:
        flat = [str(c).strip() for c in cols]
    return [c.lower() for c in flat]

def _parse_snapshot_table(soup: BeautifulSoup, limit=200) -> List[Dict]:
    """Try pandas.read_html first, with robust header matching."""
    tbl = soup.find("table")
    if not tbl:
        return []
    # read_html can return multiple dfs; pick the widest
    dfs = pd.read_html(str(tbl))
    if not dfs:
        return []
    df = max(dfs, key=lambda d: d.shape[1])
    cols = _flatten_cols(df.columns)
    # find candidate column indices
    def find_col(needle_opts):
        for i, c in enumerate(cols):
            for n in needle_opts:
                if n in c:
                    return i
        return None
    i_sym  = find_col(["symbol"])
    i_name = find_col(["name"])
    i_rank = find_col(["rank", "#"])
    if i_sym is None:
        return []
    take = df.iloc[:limit, :]
    rows=[]
    for idx, row in take.iterrows():
        try:
            sym = str(row.iloc[i_sym]).upper().strip()
            if not re.fullmatch(r"[A-Z0-9.\-]{2,10}", sym):
                continue
            name = str(row.iloc[i_name]).strip() if i_name is not None else ""
            rank = int(row.iloc[i_rank]) if (i_rank is not None and pd.notnull(row.iloc[i_rank])) else (idx+1)
            rows.append({"rank": rank, "name": name, "symbol": sym})
        except Exception:
            continue
    return rows

def _parse_snapshot_fallback(html: str, limit=200) -> List[Dict]:
    """
    Fallback parser: scan HTML for “coin cards”/anchors and extract SYMBOL heuristically.
    Works because snapshots render server-side with symbol tokens repeated near rank rows.
    """
    # common pattern: >BTC</a> or >BTC< within short spans; avoid long alphanum chains
    candidates = re.findall(r">([A-Z0-9]{2,10})<", html)
    # prune obvious non-symbols
    bad = {"USD","US","CMC","ETCETERA"}
    out=[]
    seen=set()
    for tok in candidates:
        if tok in bad: continue
        if not re.fullmatch(r"[A-Z0-9.\-]{2,10}", tok): continue
        if tok in seen: continue
        seen.add(tok)
        out.append(tok)
        if len(out) >= limit:
            break
    return [{"rank": i+1, "name": "", "symbol": s} for i, s in enumerate(out)]

# --- drop-in replacement for fetch_cmc_top_symbols (adds pagination) ---

def fetch_cmc_top_symbols(snapshot_date: datetime, limit=300, timeout=20) -> List[Dict]:
    """
    Scrape the CoinMarketCap 'Historical Snapshot' for `snapshot_date` (UTC),
    following pagination via ?start=1,101,201,... until `limit` symbols collected.
    Returns: list of dicts [{rank, name, symbol, snapshot_date}, ...]
    """
    ymd = snapshot_date.strftime("%Y%m%d")
    base = f"{CMC_HISTORICAL_ROOT}/{ymd}/"
    headers = {"User-Agent": UA}
    page_size = 100  # snapshots render 100 per page
    results: List[Dict] = []
    seen = set()

    for start_idx in range(1, limit + 1, page_size):
        url = f"{base}?start={start_idx}"
        LOG.info("CMC snapshot page %s  start=%d  %s", snapshot_date.date().isoformat(), start_idx, url)

        # request with a tiny retry
        ok = False
        for _ in range(3):
            try:
                r = requests.get(url, headers=headers, timeout=timeout)
                r.raise_for_status()
                html = r.text
                soup = BeautifulSoup(html, "html.parser")
                ok = True
                break
            except Exception as e:
                LOG.warning("retrying CMC page start=%d due to %s", start_idx, e)
                time.sleep(1.0)
        if not ok:
            break

        # try table parser, fallback to heuristic
        rows = _parse_snapshot_table(soup, limit=page_size)
        if not rows:
            LOG.warning("table parse failed; using fallback heuristic for start=%d", start_idx)
            syms = _parse_snapshot_fallback(html, limit=page_size)
            # normalize structure
            rows = [{"rank": None, "name": "", "symbol": r["symbol"]} for r in syms]

        # stitch into global list, assign proper rank = start_idx + position
        added = 0
        for j, row in enumerate(rows):
            sym = row["symbol"].upper().strip()
            if sym in seen:
                continue
            seen.add(sym)
            rank = start_idx + j
            results.append({
                "rank": rank,
                "name": row.get("name", ""),
                "symbol": sym,
                "snapshot_date": snapshot_date.date().isoformat(),
            })
            added += 1
            if len(results) >= limit:
                break

        LOG.info("collected so far: %d (added %d from this page)", len(results), added)
        if added == 0:  # page empty/blocked
            break
        if len(results) >= limit:
            break

    return results[:limit]


def build_exchange(venue="binance"):
    if venue not in ("binance","binanceusdm"):
        raise ValueError("venue must be binance|binanceusdm")
    ex = getattr(ccxt, venue)({"enableRateLimit": True, "options": {"adjustForTimeDifference": True}})
    ex.load_markets()
    return ex

def pick_market_symbol(ex, base_symbol, venue, quote="USDT") -> Optional[str]:
    bsym = SYMBOL_NORMALIZE.get(base_symbol, base_symbol).upper()
    if venue == "binance":
        cands = [f"{bsym}/{quote}"]
    else:
        cands = [f"{bsym}/{quote}:USDT", f"{bsym}/{quote}"]
    for s in cands:
        m = ex.markets.get(s)
        if not m: continue
        if venue == "binance" and m.get("spot"): return s
        if venue == "binanceusdm" and (m.get("swap") or m.get("contract")): return s
    # fallback: scan by base & quote
    for m in ex.markets.values():
        if m.get("base") == bsym:
            if venue == "binance" and m.get("spot") and m.get("quote") == quote:
                return m["symbol"]
            if venue == "binanceusdm" and (m.get("swap") or m.get("contract")) and quote in m["symbol"]:
                return m["symbol"]
    return None

def fetch_ohlcv_month(ex, symbol, start: datetime, end: datetime, timeframe="1d"):
    start_ms = int(start.timestamp()*1000)
    end_ms   = int(end.timestamp()*1000) - 1
    out=[]
    cursor = start_ms
    while True:
        params={}
        if ex.id in ("binance","binanceusdm"):
            params["endTime"] = end_ms
        chunk = ex.fetch_ohlcv(symbol, timeframe=timeframe, since=cursor, limit=1500, params=params)
        if not chunk: break
        out.extend(chunk)
        last_ts = chunk[-1][0]
        if last_ts >= end_ms: break
        cursor = last_ts + 60_000
        time.sleep(ex.rateLimit/1000)
    # strict filter to [start, end)
    out = [row for row in out if start_ms <= row[0] <= end_ms]
    return out



def fetch_top200_months(months=12, venue="binance", quote="USDT", limit=200, verbose=True):
    now = datetime.now(timezone.utc)
    bounds = month_bounds_utc(now, months)
    ex = build_exchange(venue)
    results = {}
    top_lists = {}
    for (mstart, mend) in bounds:
        yyyymm = mstart.strftime("%Y-%m")
        snap = first_cmc_snapshot_on_or_after(mstart)
        top = fetch_cmc_top_symbols(snap, limit=limit)
        top_df = pd.DataFrame(top)
        top_lists[yyyymm] = top_df
        if verbose:
            LOG.info("[%s] ranked symbols=%d", yyyymm, len(top_df))
        mapped=[]
        for row in top:
            sym = row["symbol"].upper()
            msym = pick_market_symbol(ex, sym, venue, quote=quote)
            if msym:
                mapped.append({**row, "exchange_symbol": msym})
        if verbose:
            LOG.info("[%s] tradable on %s: %d", yyyymm, ex.id, len(mapped))
        recs=[]
        for i, row in enumerate(mapped, 1):
            msym = row["exchange_symbol"]
            try:
                candles = fetch_ohlcv_month(ex, msym, mstart, mend, "1d")
                for ts,o,h,l,c,v in candles:
                    recs.append({
                        "rank": row["rank"], "symbol": row["symbol"], "name": row.get("name",""),
                        "exchange": ex.id, "exchange_symbol": msym,
                        "time": datetime.utcfromtimestamp(ts/1000).replace(tzinfo=timezone.utc).isoformat(),
                        "open": o, "high": h, "low": l, "close": c, "volume": v,
                        "month_start": mstart.date().isoformat(),
                        "snapshot_date": row["snapshot_date"],
                    })
                if i % 20 == 0 and verbose:
                    LOG.info("[%s] %d/%d markets done", yyyymm, i, len(mapped))
            except ccxt.BaseError as e:
                LOG.warning("[%s] fetch failed %s: %s", yyyymm, msym, e)
        results[yyyymm] = pd.DataFrame(recs)
        if verbose:
            LOG.info("[%s] candles: %d", yyyymm, len(results[yyyymm]))
    return results, top_lists


In [7]:
dfs, tops = fetch_top200_months(months=12, venue="binance")   # or venue="binanceusdm"
for m, df in dfs.items():
    print(m, df.shape, "tradable:", df["exchange_symbol"].nunique() if not df.empty else 0)
    # peek
    display(df.head())


2025-10-01 17:38:51,780 INFO CMC snapshot page 2024-11-03  start=1  https://coinmarketcap.com/historical/20241103/?start=1
C:\Users\nahid\AppData\Local\Temp\ipykernel_30088\3426699124.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(str(tbl))
2025-10-01 17:38:52,630 WARNING table parse failed; using fallback heuristic for start=1
2025-10-01 17:38:52,630 INFO collected so far: 53 (added 53 from this page)
2025-10-01 17:38:52,630 INFO CMC snapshot page 2024-11-03  start=101  https://coinmarketcap.com/historical/20241103/?start=101
C:\Users\nahid\AppData\Local\Temp\ipykernel_30088\3426699124.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(str(tbl))
2025-10-01 17:38:53,180 WARNING table parse faile

2024-11 (750, 13) tradable: 25


,rank,symbol,name,exchange,exchange_symbol,time,open,high,low,close,volume,month_start,snapshot_date
0,2,BTC,,binance,BTC/USDT,2024-11-01T00:00:00+00:00,70292.01,71632.95,68820.14,69496.01,38301.86755,2024-11-01,2024-11-03
1,2,BTC,,binance,BTC/USDT,2024-11-02T00:00:00+00:00,69496.00,69914.37,69000.14,69374.74,10521.67243,2024-11-01,2024-11-03
2,2,BTC,,binance,BTC/USDT,2024-11-03T00:00:00+00:00,69374.74,69391.00,67478.73,68775.99,24995.70243,2024-11-01,2024-11-03
3,2,BTC,,binance,BTC/USDT,2024-11-04T00:00:00+00:00,68775.99,69500.00,66835.00,67850.01,29800.39187,2024-11-01,2024-11-03
4,2,BTC,,binance,BTC/USDT,2024-11-05T00:00:00+00:00,67850.01,70577.91,67476.63,69372.01,33355.06888,2024-11-01,2024-11-03


2024-12 (806, 13) tradable: 26


,rank,symbol,name,exchange,exchange_symbol,time,open,high,low,close,volume,month_start,snapshot_date
0,2,BTC,,binance,BTC/USDT,2024-12-01T00:00:00+00:00,96407.99,97836.00,95693.88,97185.18,16938.604520,2024-12-01,2024-12-01
1,2,BTC,,binance,BTC/USDT,2024-12-02T00:00:00+00:00,97185.17,98130.00,94395.00,95840.62,37958.669810,2024-12-01,2024-12-01
2,2,BTC,,binance,BTC/USDT,2024-12-03T00:00:00+00:00,95840.61,96305.52,93578.17,95849.69,35827.322830,2024-12-01,2024-12-01
3,2,BTC,,binance,BTC/USDT,2024-12-04T00:00:00+00:00,95849.69,99000.00,94587.83,98587.32,43850.537280,2024-12-01,2024-12-01
4,2,BTC,,binance,BTC/USDT,2024-12-05T00:00:00+00:00,98587.32,104088.00,90500.00,96945.63,109921.729662,2024-12-01,2024-12-01


...

,rank,symbol,name,exchange,exchange_symbol,time,open,high,low,close,volume,month_start,snapshot_date
0,2,BTC,,binance,BTC/USDT,2025-08-01T00:00:00+00:00,115764.07,116052.00,112722.58,113297.93,24487.10206,2025-08-01,2025-08-03
1,2,BTC,,binance,BTC/USDT,2025-08-02T00:00:00+00:00,113297.92,114063.49,112003.00,112546.35,11507.33428,2025-08-01,2025-08-03
2,2,BTC,,binance,BTC/USDT,2025-08-03T00:00:00+00:00,112546.35,114799.97,111920.00,114208.80,7397.76046,2025-08-01,2025-08-03
3,2,BTC,,binance,BTC/USDT,2025-08-04T00:00:00+00:00,114208.81,115720.00,114107.60,115055.03,9667.67916,2025-08-01,2025-08-03
4,2,BTC,,binance,BTC/USDT,2025-08-05T00:00:00+00:00,115055.03,115127.81,112650.00,114129.75,12042.79078,2025-08-01,2025-08-03


2025-09 (540, 13) tradable: 18


,rank,symbol,name,exchange,exchange_symbol,time,open,high,low,close,volume,month_start,snapshot_date
0,2,BTC,,binance,BTC/USDT,2025-09-01T00:00:00+00:00,108246.36,109912.40,107255.00,109237.42,16053.60219,2025-09-01,2025-09-07
1,2,BTC,,binance,BTC/USDT,2025-09-02T00:00:00+00:00,109237.43,111771.52,108393.39,111240.01,18510.28756,2025-09-01,2025-09-07
2,2,BTC,,binance,BTC/USDT,2025-09-03T00:00:00+00:00,111240.01,112575.27,110528.71,111705.71,11773.72084,2025-09-01,2025-09-07
3,2,BTC,,binance,BTC/USDT,2025-09-04T00:00:00+00:00,111705.72,112180.00,109329.12,110730.87,12203.13536,2025-09-01,2025-09-07
4,2,BTC,,binance,BTC/USDT,2025-09-05T00:00:00+00:00,110730.87,113384.62,110206.96,110659.99,21587.40888,2025-09-01,2025-09-07


2025-10 (2, 13) tradable: 2


,rank,symbol,name,exchange,exchange_symbol,time,open,high,low,close,volume,month_start,snapshot_date
0,4,BTC,,binance,BTC/USDT,2025-10-01T00:00:00+00:00,114048.94,118199.0,113966.67,118025.58,15237.89066,2025-10-01,2025-10-05
1,5,ETH,,binance,ETH/USDT,2025-10-01T00:00:00+00:00,4145.15,4344.0,4123.08,4335.51,390669.92290,2025-10-01,2025-10-05


In [1]:
dfs.shape

NameError: name 'dfs' is not defined

In [8]:
dfs['2024-10'].loc[dfs['2024-10']['symbol']=='BTC']

KeyError: '2024-10'

In [6]:
# see the size of dfs and tops
print("dfs keys:", list(dfs.keys()))
print("tops keys:", list(tops.keys()))

dfs keys: ['2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06', '2025-07', '2025-08', '2025-09', '2025-10']
tops keys: ['2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06', '2025-07', '2025-08', '2025-09', '2025-10']


In [9]:
# I want to save dfs and tops to csv files so that I can load them later without re-fetching
import os
os.makedirs("data", exist_ok=True)
for m, df in dfs.items():
    df.to_csv(f"data/ohlcv_{m}.csv", index=False)
for m, df in tops.items():
    df.to_csv(f"data/top200_{m}.csv", index=False)

In [10]:
# Load saved data
import pandas as pd
import os
data_dir = "data"

for fname in os.listdir(data_dir):
    if fname.startswith("ohlcv_") and fname.endswith(".csv"):
        m = fname[len("ohlcv_"):-len(".csv")]
        path = os.path.join(data_dir, fname)
        df = pd.read_csv(path)
        dfs[m] = df
        print(f"Loaded {m}: {df.shape}")
    elif fname.startswith("top200_") and fname.endswith(".csv"):
        m = fname[len("top200_"):-len(".csv")]
        path = os.path.join(data_dir, fname)
        df = pd.read_csv(path)
        tops[m] = df
        print(f"Loaded top {m}: {df.shape}")

Loaded 2024-11: (750, 13)
Loaded 2024-12: (806, 13)
Loaded 2025-01: (775, 13)
Loaded 2025-02: (644, 13)
Loaded 2025-03: (744, 13)
Loaded 2025-04: (660, 13)
Loaded 2025-05: (646, 13)
Loaded 2025-06: (600, 13)
Loaded 2025-07: (620, 13)
Loaded 2025-08: (620, 13)
Loaded 2025-09: (540, 13)
Loaded 2025-10: (2, 13)
Loaded top 2024-11: (53, 4)
Loaded top 2024-12: (53, 4)
Loaded top 2025-01: (50, 4)
Loaded top 2025-02: (51, 4)
Loaded top 2025-03: (50, 4)
Loaded top 2025-04: (49, 4)
Loaded top 2025-05: (50, 4)
Loaded top 2025-06: (49, 4)
Loaded top 2025-07: (50, 4)
Loaded top 2025-08: (50, 4)
Loaded top 2025-09: (50, 4)
Loaded top 2025-10: (8, 4)


In [11]:
# --- paste to replace CG networking + prefetch (adds strict timeouts, attempt logs, jitter, and no double logs) ---

import time, json, math, hashlib, random, requests, logging
from pathlib import Path
from datetime import datetime, timezone

from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# logger: prevent duplicate lines across re-execs
LOG = logging.getLogger("cg_top200_nb")
if not getattr(LOG, "_configured", False):
    h = logging.StreamHandler()
    h.setFormatter(logging.Formatter("%(asctime)s %(levelname)s %(message)s"))
    LOG.addHandler(h)
    LOG.setLevel(logging.INFO)
    LOG.propagate = False
    LOG._configured = True

CG_BASE = "https://api.coingecko.com/api/v3"
CG_CACHE_DIR = Path("./.cg_cache"); CG_CACHE_DIR.mkdir(parents=True, exist_ok=True)

# conservative defaults (tune here)
CONNECT_TIMEOUT = 5          # seconds
READ_TIMEOUT    = 12         # seconds (keeps single read under ~12s)
MAX_ATTEMPTS    = 4
BACKOFF_BASE    = 0.8        # exponential backoff base
POLITE_DELAY    = 0.45       # base inter-request delay
SLOW_LOG_SEC    = 5.0        # log when a single attempt exceeds this
USE_DAILY_ENDPOINT = True    # True => /market_chart?days=… (usually faster/cached)

# requests session + retries for transient 5xx (we do manual retry loop too)
sess = requests.Session()
adapter = HTTPAdapter(max_retries=Retry(total=0))  # disable urllib3 auto-retries; we do our own so it’s visible
sess.mount("https://", adapter)
sess.headers.update({"User-Agent": "cg-top200-notebook/1.1"})

def _cache_path(prefix: str, key: str) -> Path:
    h = hashlib.sha256(key.encode()).hexdigest()[:32]
    return CG_CACHE_DIR / f"{prefix}_{h}.json"

def cache_get(prefix: str, key: str):
    p = _cache_path(prefix, key)
    if p.exists():
        try:
            return json.loads(p.read_text())
        except Exception:
            return None
    return None

def cache_put(prefix: str, key: str, obj):
    _cache_path(prefix, key).write_text(json.dumps(obj))

def cg_get(path: str, params: dict, *, attempts=MAX_ATTEMPTS):
    """GET with strict per-attempt timeouts, explicit retry logs, and 429 handling."""
    url = f"{CG_BASE}{path}"
    key = url + "?" + "&".join(f"{k}={v}" for k,v in sorted(params.items()))
    cached = cache_get("cg", key)
    if cached is not None:
        LOG.debug("CG cache hit %s", path)
        return cached

    for k in range(1, attempts + 1):
        # jitter to avoid global thundering herd
        time.sleep(POLITE_DELAY + random.uniform(0, 0.25))
        t0 = time.perf_counter()
        try:
            LOG.info("CG attempt %d/%d %s params=%s", k, attempts, path, params)
            r = sess.get(url, params=params, timeout=(CONNECT_TIMEOUT, READ_TIMEOUT))
            dt = time.perf_counter() - t0
            if dt > SLOW_LOG_SEC:
                LOG.warning("CG SLOW %s dt=%.2fs", path, dt)
            if r.status_code == 429:
                ra = r.headers.get("Retry-After")
                sleep_s = float(ra) if ra and str(ra).isdigit() else BACKOFF_BASE * (2 ** (k - 1))
                LOG.warning("CG 429 %s; sleeping %.2fs then retry", path, sleep_s)
                time.sleep(sleep_s)
                continue
            if 500 <= r.status_code < 600:
                LOG.warning("CG %s %s; retrying", path, r.status_code)
                time.sleep(BACKOFF_BASE * (2 ** (k - 1)))
                continue
            r.raise_for_status()
            data = r.json()
            cache_put("cg", key, data)
            LOG.info("CG ok %s status=%s dt=%.2fs", path, r.status_code, dt)
            return data
        except (requests.Timeout, requests.ConnectionError) as e:
            dt = time.perf_counter() - t0
            LOG.warning("CG TIMEOUT/CONN %s after %.2fs (attempt %d/%d): %s", path, dt, k, attempts, str(e)[:160])
            time.sleep(BACKOFF_BASE * (2 ** (k - 1)))
        except requests.RequestException as e:
            LOG.warning("CG ERROR %s (attempt %d/%d): %s", path, k, attempts, str(e)[:160])
            time.sleep(BACKOFF_BASE * (2 ** (k - 1)))
    raise RuntimeError(f"CG GET failed after {attempts} attempts: {path} params={params}")

def cg_top_now(n: int = 600, vs="usd"):
    pages = math.ceil(n / 250)
    rows, rank = [], 1
    for p in range(1, pages + 1):
        per = 250 if p < pages else (n - 250 * (pages - 1))
        per = max(per, 1)
        data = cg_get("/coins/markets", {
            "vs_currency": vs, "order": "market_cap_desc",
            "per_page": per, "page": p, "sparkline": "false",
        })
        for d in data:
            rows.append({"id": d["id"], "symbol": str(d["symbol"]).upper(), "name": d["name"], "rank_now": rank})
            rank += 1
        LOG.info("top_now progress: %d/%d", len(rows), n)
    df = pd.DataFrame(rows)
    LOG.info("top_now gathered: %d", len(df))
    return df

def cg_market_caps_series(coin_id: str, start_dt: datetime, end_dt: datetime, vs="usd"):
    """Prefer daily series endpoint (faster, cached); fallback to range."""
    if USE_DAILY_ENDPOINT:
        days = max(1, int((end_dt - start_dt).days) + 6)
        data = cg_get(f"/coins/{coin_id}/market_chart", {
            "vs_currency": vs, "days": days, "interval": "daily"
        })
        return (data.get("market_caps") or [])
    else:
        data = cg_get(f"/coins/{coin_id}/market_chart/range", {
            "vs_currency": vs, "from": int(start_dt.replace(tzinfo=timezone.utc).timestamp()),
            "to": int(end_dt.replace(tzinfo=timezone.utc).timestamp())
        })
        return (data.get("market_caps") or [])

def prefetch_mcap_ranges(coin_ids, start_dt, end_dt, vs="usd", log_every=25):
    series = {}
    N = len(coin_ids)
    LOG.info("prefetch market_caps for %d coins [%s → %s] (daily=%s)", N, start_dt.date(), end_dt.date(), USE_DAILY_ENDPOINT)
    for i, cid in enumerate(coin_ids, 1):
        t0 = time.perf_counter()
        try:
            series[cid] = cg_market_caps_series(cid, start_dt, end_dt, vs=vs)
        except Exception as e:
            LOG.warning("prefetch FAIL id=%s (%d/%d): %s", cid, i, N, str(e)[:160])
            series[cid] = []
        dt = time.perf_counter() - t0
        if i % log_every == 0 or dt > SLOW_LOG_SEC:
            LOG.info("prefetch %d/%d id=%s caps_pts=%d dt=%.2fs", i, N, cid, len(series[cid]), dt)
    return series

def cap_on_or_near(date_dt, caps_series, max_lag_days: int = 5):
    if not caps_series: return None
    target = int(date_dt.replace(tzinfo=timezone.utc).timestamp()) * 1000
    best = min(caps_series, key=lambda x: abs(x[0] - target))
    if abs(best[0] - target) > max_lag_days * 86400_000:
        return None
    return float(best[1]) if best and best[1] is not None else None


# ----------------- Exchange + OHLCV -------
def build_exchange(venue="binance"):
    if venue not in ("binance", "binanceusdm"):
        raise ValueError("venue must be binance|binanceusdm")
    ex = getattr(ccxt, venue)({"enableRateLimit": True, "options": {"adjustForTimeDifference": True}})
    ex.load_markets()
    LOG.info("loaded markets for %s: %d", ex.id, len(ex.markets))
    return ex

def pick_market_symbol(ex, base_symbol: str, venue: str, quote="USDT") -> Optional[str]:
    bsym = SYMBOL_NORMALIZE.get(base_symbol.upper(), base_symbol.upper())
    cands = [f"{bsym}/{quote}"] if venue == "binance" else [f"{bsym}/{quote}:USDT", f"{bsym}/{quote}"]
    for s in cands:
        m = ex.markets.get(s)
        if not m: continue
        if venue == "binance" and m.get("spot"): return s
        if venue == "binanceusdm" and (m.get("swap") or m.get("contract")): return s
    for m in ex.markets.values():
        if m.get("base") == bsym:
            if venue == "binance" and m.get("spot") and m.get("quote") == quote: return m["symbol"]
            if venue == "binanceusdm" and (m.get("swap") or m.get("contract")) and quote in m["symbol"]: return m["symbol"]
    return None

def fetch_ohlcv_month(ex, symbol: str, start: datetime, end: datetime, timeframe="1d") -> List[List]:
    start_ms = int(start.timestamp() * 1000)
    end_ms   = int(end.timestamp() * 1000) - 1
    out = []
    cursor = start_ms
    LOG.info("OHLCV %s %s [%s → %s]", ex.id, symbol, datetime.utcfromtimestamp(start_ms/1000).date(), datetime.utcfromtimestamp((end_ms+1)/1000).date())
    while True:
        params = {}
        if ex.id in ("binance", "binanceusdm"): params["endTime"] = end_ms
        t0 = time.perf_counter()
        chunk = ex.fetch_ohlcv(symbol, timeframe=timeframe, since=cursor, limit=1500, params=params)
        dt = time.perf_counter() - t0
        LOG.debug("fetch_ohlcv chunk n=%d cursor=%s dt=%.2fs", len(chunk), cursor, dt)
        if not chunk: break
        out.extend(chunk)
        last_ts = chunk[-1][0]
        if last_ts >= end_ms: break
        cursor = last_ts + 60_000
        time.sleep(ex.rateLimit / 1000)
    out = [row for row in out if start_ms <= row[0] <= end_ms]
    LOG.info("OHLCV got %d rows for %s", len(out), symbol)
    return out

# ----------------- main driver ------------
def fetch_monthly_top200_ohlcv(
    months: int = 12,
    venue: str = "binance",
    quote: str = "USDT",
    top_now_n: int = 600,
    per_month_k: int = 200,
    vs_currency: str = "usd",
    verbose: bool = True,
    rank_only: bool = False,
    max_lag_days: int = 5,
):
    """
    1) Pull today's top_now_n from CG (coins/markets, paginated).
    2) Prefetch market_caps once per coin over the full [first-3d, last+3d] span.
    3) Rank each month by nearest cap to month start; take top per_month_k.
    4) If rank_only=False, fetch month OHLCV from ccxt for the mapped markets.
    """
    now = datetime.now(timezone.utc)
    bounds = month_bounds_utc(now, months)
    first_start = bounds[0][0] - timedelta(days=3)
    last_end    = bounds[-1][1] + timedelta(days=3)

    # 1) candidates
    cand = cg_top_now(n=top_now_n, vs=vs_currency)

    # 2) prefetch series
    caps_series = prefetch_mcap_ranges(cand["id"].tolist(), first_start, last_end, vs=vs_currency)

    # 3) rank per month
    cohorts_by_month: Dict[str, pd.DataFrame] = {}
    for (mstart, mend) in bounds:
        yyyymm = mstart.strftime("%Y-%m")
        LOG.info("[%s] ranking by market cap at %sZ", yyyymm, mstart.isoformat())
        rows = []
        miss = 0
        for _, r in cand.iterrows():
            caps = caps_series.get(r["id"], [])
            cap  = cap_on_or_near(mstart, caps, max_lag_days=max_lag_days)
            if cap is None:
                miss += 1
                continue
            rows.append({"cg_id": r["id"], "cg_symbol": r["symbol"], "cg_name": r["name"], "mcap_month_start": cap})
        cohort = pd.DataFrame(rows)
        cohort.sort_values("mcap_month_start", ascending=False, inplace=True, ignore_index=True)
        cohort["rank_month"] = cohort.index + 1
        cohort = cohort.head(per_month_k)
        LOG.info("[%s] cohort size=%d (missing caps=%d)", yyyymm, len(cohort), miss)
        cohorts_by_month[yyyymm] = cohort

    if rank_only:
        return {m: pd.DataFrame() for m in cohorts_by_month}, cohorts_by_month

    # 4) OHLCV
    ex = build_exchange(venue)
    ohlcv_by_month: Dict[str, pd.DataFrame] = {}
    for (mstart, mend) in bounds:
        yyyymm = mstart.strftime("%Y-%m")
        cohort = cohorts_by_month[yyyymm]
        LOG.info("[%s] mapping to %s markets", yyyymm, ex.id)
        mapped = []
        for _, rr in cohort.iterrows():
            msym = pick_market_symbol(ex, rr["cg_symbol"], venue, quote=quote)
            if msym:
                mapped.append({**rr.to_dict(), "exchange": ex.id, "exchange_symbol": msym})
        LOG.info("[%s] tradable markets=%d", yyyymm, len(mapped))

        recs = []
        for i, rr in enumerate(mapped, 1):
            LOG.info("[%s] (%d/%d) OHLCV %s", yyyymm, i, len(mapped), rr["exchange_symbol"])
            try:
                cs = fetch_ohlcv_month(ex, rr["exchange_symbol"], mstart, mend, "1d")
                for t,o,h,l,c,v in cs:
                    recs.append({
                        "rank_month": int(rr["rank_month"]),
                        "cg_id": rr["cg_id"], "cg_symbol": rr["cg_symbol"], "cg_name": rr["cg_name"],
                        "exchange": rr["exchange"], "exchange_symbol": rr["exchange_symbol"],
                        "time": datetime.utcfromtimestamp(t/1000).replace(tzinfo=timezone.utc).isoformat(),
                        "open": o, "high": h, "low": l, "close": c, "volume": v,
                        "month_start": mstart.date().isoformat(),
                    })
            except ccxt.BaseError as e:
                LOG.warning("[%s] fetch failed %s: %s", yyyymm, rr["exchange_symbol"], str(e)[:160])
        ohlcv_by_month[yyyymm] = pd.DataFrame(recs)

    return ohlcv_by_month, cohorts_by_month


In [ ]:
# dial up verbosity if you want wire-level detail
LOG.setLevel(logging.DEBUG)

# fast sanity: rank only
_, cohorts = fetch_monthly_top200_ohlcv(months=3, rank_only=True, top_now_n=400)

# full pull (start small to validate)
ohlcv, cohorts = fetch_monthly_top200_ohlcv(months=3, venue="binance", top_now_n=400, per_month_k=200)

for m, df in ohlcv.items():
    print(m, df.shape, "uniq markets:", df["exchange_symbol"].nunique() if not df.empty else 0)
    display(cohorts[m].head(10))
    display(df.head())


2025-10-01 17:43:51,829 DEBUG CG cache hit /coins/markets
2025-10-01 17:43:51,830 INFO top_now progress: 250/400


2025-10-01 17:43:51,849 DEBUG CG cache hit /coins/markets
2025-10-01 17:43:51,850 INFO top_now progress: 400/400
2025-10-01 17:43:51,852 INFO top_now gathered: 400
2025-10-01 17:43:51,853 INFO prefetch market_caps for 400 coins [2025-07-29 → 2025-11-04] (daily=True)
2025-10-01 17:43:51,870 DEBUG CG cache hit /coins/bitcoin/market_chart
2025-10-01 17:43:51,884 DEBUG CG cache hit /coins/ethereum/market_chart
2025-10-01 17:43:51,900 DEBUG CG cache hit /coins/ripple/market_chart
2025-10-01 17:43:51,916 DEBUG CG cache hit /coins/tether/market_chart
2025-10-01 17:43:51,933 DEBUG CG cache hit /coins/binancecoin/market_chart
2025-10-01 17:43:51,950 DEBUG CG cache hit /coins/solana/market_chart
2025-10-01 17:43:51,970 DEBUG CG cache hit /coins/usd-coin/market_chart
2025-10-01 17:43:51,992 DEBUG CG cache hit /coins/staked-ether/market_chart
2025-10-01 17:43:52,015 DEBUG CG cache hit /coins/dogecoin/market_chart
2025-10-01 17:43:52,050 DEBUG CG cache hit /coins/tron/market_chart
2025-10-01 17:43: